In [2]:
#python packages

#Bert Model
from transformers import pipeline, AutoTokenizer, BertForMaskedLM
unmasker = pipeline('fill-mask', model='dumitrescustefan/bert-base-romanian-cased-v1', top_k=10)

#tokenizer
import spacy
from spacy import displacy
NLP = spacy.load("ro_core_news_lg")

#text classification
import fasttext.util
fasttext.util.download_model('ro', if_exists='ignore')
ft = fasttext.load_model('cc.ro.300.bin')

#zipf_score
from wordfreq import zipf_frequency

#cosine similarity
import scipy.spatial, scipy.special

#min max scaler
from sklearn.preprocessing import MinMaxScaler

C:\Users\Anghel Fabian\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json

synonyms_dict = {}
antonyms_dict = {}

#load synonyms and antonyms
#with open(join(dirname(__file__), 'synonyms.json'), 'r') as f:
with open('scraping/synonyms.json', 'r') as f:
    synonyms_dict = json.load(f)

#with open(join(dirname(__file__), 'antonyms.json'), 'r') as f:
with open('scraping/antonyms.json', 'r') as f:
    antonyms_dict = json.load(f)

In [177]:
#load dex online data
with open('dex-online-database/word_inflections.json', 'r') as f:
    word_inflections = json.load(f)

with open('dex-online-database/spaCy_tags.json', 'r') as f:
    spacy_tags = json.load(f)

with open('dex-online-database/pron_and_num_pairs.json', 'r', encoding='utf-8') as f:
    pron_pairs = json.load(f)

In [178]:
def preprocess_sentence(sentence):
    #get right versions of ș/ț
    sentence = sentence.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

    #transform a/i + unicode 770/u+0302 in â/î
    sentence = sentence.replace("a\u0302", "â").replace("i\u0302", "î").replace("A\u0302", "Â").replace("I\u0302", "Î")

    #transform s/t + unicode 807/u+0327 or 806/u+0326 in ș/ț
    sentence = sentence.replace("s\u0327", "ș").replace("t\u0327", "ț").replace("S\u0327", "Ș").replace("T\u0327", "Ț")
    sentence = sentence.replace("s\u0326", "ș").replace("t\u0326", "ț").replace("S\u0326", "Ș").replace("T\u0326", "Ț")

    #transform a + unicode 774/u+0306 in ă
    sentence = sentence.replace("a\u0306", "ă").replace("A\u0307", "Ă")

    return sentence

In [195]:
def valid_token(token, zipf_score):
    #if token is punctuation
    if token.is_punct:
        return False
    
    #if token is number
    if token.is_digit or token.like_num:
        return False
    
    #if token is whitespace
    if token.is_space:
        return False
    
    #if token is currency:
    if token.is_currency:
        return False
    
    #if token is url or email
    if token.like_url or token.like_email:
        return False
    
    #should be replaced only tokens that are nouns, adjectives, adverbs or verbs (proper nouns are also excluded here)
    if token.pos_ not in ["NOUN", "ADJ", "ADV", "VERB", "AUX"]:
        return False
    
    #if token is proper noun
    if token.pos_ == "NOUN" and token.tag_[1] == 'p':
        return False
    
    #exclude capitalised words if not at the start of a sentence (it's a name)
    if not token.is_sent_start and token.text[0].isupper():
        return False

    #exclude cardinal directions
    cardinal_directions = ["nord", "est", "sud", "vest", "nord-est", "nord-vest", "sud-est", "sud-vest", "nordic", "estic", "sudic", "vestic", "nord-estic", "nord-vestic", "sud-estic", "sud-vestic"]
    if token.lemma_ in cardinal_directions:
        return False

    #exclude stop words (should not be replaced as they are frequently used words)
    if token.is_stop:
        return False
    
    #too high zipf score
    #if zipf_score > 4.25:
    if zipf_score > 4.5:
    #if zipf_score > 4:
        return False
    
    #to do: check ENTs
    #some can be general words (like "îmbrăcăminte", "litoral", "alcool")
    #DATETIME should still be replaced
    
    return True

In [196]:
def word_candidates(sentence, new_tokens):
    tokens = NLP(sentence)
    replacable_tokens = []

    #sort tokens by Zipf score
    for token in tokens:
        #token_score = zipf_frequency(token.text, 'ro')
        token_score = zipf_frequency(token.lemma_, "ro")
        if valid_token(token, token_score) and token.text not in new_tokens:
            #context = extract_context(tokens, token.i)
            replacable_tokens.append((token, token_score))

    replacable_tokens.sort(key=lambda x: x[1])

    return replacable_tokens

In [245]:
def noun_substitution_generation(sentence, token):
    masked_sentence = sentence[:token.idx] + "[MASK]" + sentence[token.idx + len(token.text):]
    model_sentence = "[CLS] " + sentence + " [SEP] " + masked_sentence + " [SEP]"
    result = unmasker(model_sentence)
    suggestions = []

    can_change_form = True
    changed_tokens_list = []

    sentence_tokens = NLP(sentence)

    token_children = [child.i for child in token.children]
    token_ancestors = [ancestor.i for ancestor in token.ancestors]

    #change gender for related words in the sentence
    for sent_token in sentence_tokens:
        changed = False
        if sent_token.i in token_children or sent_token.i in token_ancestors:
            #if word is adjective search if opposite gender is available
            if sent_token.pos_ == "ADJ":
                if sent_token.tag_ in spacy_tags:
                    tag_id = spacy_tags[sent_token.tag_]["dex_online_id"][0]
                    if sent_token.tag_[3] == "m":
                        tag_id += 8
                    else:
                        tag_id -= 8
                    tag_id = str(tag_id)

                    if sent_token.lemma_ in word_inflections:
                        if tag_id in word_inflections[sent_token.lemma_]["inflections"]:
                            changed_tokens_list.append(word_inflections[sent_token.lemma_]["inflections"][tag_id])
                            changed = True
            
            #if in list get opposite gender
            elif sent_token.text in pron_pairs:
                changed_tokens_list.append(pron_pairs[sent_token.text])
                changed = True

                #replace article if present
                for child in sent_token.children:
                    if child.text == "a":
                        changed_tokens_list[child.i] = "al"
                    elif child.text == "al":
                        changed_tokens_list[child.i] = "a"
                    elif child.text == "ai":
                        changed_tokens_list[child.i] = "ale"
                    elif child.text == "ale":
                        changed_tokens_list[child.i] = "ai"

            #if ordinal numeral change gender
            elif sent_token.pos_ == "NUM" and sent_token.tag_[1] == "o" and sent_token.lemma != "primul":
                tag_id = 45
                if sent_token.tag_[2] == "f":
                    tag_id = 41
                tag_id = str(tag_id)
                sent_token_lemma = sent_token.lemma_ + "lea"
                changed_tokens_list.append(word_inflections[sent_token_lemma]["inflections"][tag_id])
                changed = True

        #mask wanted token
        elif sent_token.i == token.i:
            changed_tokens_list.append("[MASK]")
            changed = True

        if not changed:
            changed_tokens_list.append(sent_token.text)

    #build sentence with changed genders and get suggestions
    changed_sentence = ""
    for sent_token in changed_tokens_list:
        if sent_token in ",.;!?:-_)]}":
            changed_sentence = changed_sentence.rstrip()
            changed_sentence += sent_token + " "
        elif sent_token[:-1] == "-" or sent_token in "([{":
            changed_sentence += sent_token
        else:
            changed_sentence += sent_token + " "

    changed_model_sentence = "[CLS] " + sentence + " [SEP] " + changed_sentence + " [SEP]"
    changed_result = unmasker(changed_model_sentence)
    
    #filter original suggestions
    for x in result:
        #check if suggestion is same part of speech 
        replaced_sentence = sentence[:token.idx] + x["token_str"] + sentence[token.idx + len(token.text):]
        replaced_tokens = NLP(replaced_sentence)
        suggestion_token = replaced_tokens[token.i]

        if token.pos != suggestion_token.pos and not (token.pos_ == "AUX" and suggestion_token.pos_ == "VERB") and not (token.pos_ == "VERB" and suggestion_token.pos_ == "AUX"):
            continue

        x["changed"] = False
        suggestions.append(x)

    #filter suggestiongs for changed sentence
    for x in changed_result:
        #check if suggestion is same part of speech 
        changed_tokens_list[token.i] = x["token_str"]
        replaced_sentence = " ".join(changed_tokens_list)
        replaced_tokens = NLP(replaced_sentence)
        suggestion_token = replaced_tokens[token.i]
        changed_tokens_list[token.i] = token.text

        if token.pos != suggestion_token.pos and not (token.pos_ == "AUX" and suggestion_token.pos_ == "VERB") and not (token.pos_ == "VERB" and suggestion_token.pos_ == "AUX"):
            continue

        if len(token.tag_) > 2 and len(suggestion_token.tag_) > 2 and token.tag_[2] != suggestion_token.tag_[2]:
            x["changed"] = True
        else:
            x["changed"] = False
        suggestions.append(x)

    return (suggestions, changed_sentence)
    

In [238]:
def substitution_generation(sentence, token):
    # generate a masked sentence in this form: [CLS] original sentence [SEP] masked sentence [SEP]
    masked_sentence = sentence[:token.idx] + "[MASK]" + sentence[token.idx + len(token.text):]
    model_sentence = "[CLS] " + sentence + " [SEP] " + masked_sentence + " [SEP]"
    result = unmasker(model_sentence)
    suggestions = []

    for x in result:
        #check if suggestion is same part of speech 
        replaced_sentence = sentence[:token.idx] + x["token_str"] + sentence[token.idx + len(token.text):]
        replaced_tokens = NLP(replaced_sentence)
        suggestion_token = replaced_tokens[token.i]

        if token.pos != suggestion_token.pos and not (token.pos_ == "AUX" and suggestion_token.pos_ == "VERB") and not (token.pos_ == "VERB" and suggestion_token.pos_ == "AUX"):
            continue

        x["changed"] = False
        suggestions.append(x)

    return suggestions    

In [239]:
def synonym_score(suggestion, token):
    sugg_token = NLP(suggestion)[0]
    sugg_lemma = sugg_token.lemma_
    token_lemma = token.lemma_

    #positive score if synonyms
    if (token_lemma in synonyms_dict and sugg_lemma in synonyms_dict[token_lemma]) or (sugg_lemma in synonyms_dict and token_lemma in synonyms_dict[sugg_lemma]):
        return 1

    #negative score if antonyms
    if (token_lemma in antonyms_dict and sugg_lemma in antonyms_dict[token_lemma]) or (sugg_lemma in antonyms_dict and token_lemma in antonyms_dict[sugg_lemma]):
        return 0
    if "ne" + token_lemma == sugg_lemma or "ne" + sugg_lemma == token_lemma:
        return 0
    if "in" + token_lemma == sugg_lemma or "in" + sugg_lemma == token_lemma:
        return 0
    
    #negative score if same word to encourage different suggestions
    if token_lemma == sugg_lemma:
        return 0.25
    
    #netural score if not related
    return 0.5

In [240]:
def substitution_ranking(suggetions, token):
    substitutions = []

    bert_scores = []
    zipf_scores = []
    fasttext_scores = []
    synonym_scores = []

    # calculate a score for each suggestion based on BERT, Zipf and FastText
    for x in suggetions:
        bert_scores.append([x["score"]])

        zipf_score = zipf_frequency(x["token_str"], 'ro')
        zipf_scores.append([zipf_score])

        fasttext_score = 1 - scipy.spatial.distance.cosine(ft[token.text], ft[x["token_str"]])
        fasttext_scores.append([fasttext_score])

        syn_score = synonym_score(x["token_str"], token)
        synonym_scores.append([syn_score])
    
    if len(bert_scores) > 0:
        bert_scaler = MinMaxScaler()
        zipf_scaler = MinMaxScaler()
        fasttext_scaler = MinMaxScaler()
        synonym_scaler = MinMaxScaler()

        bert_scores_norm = bert_scaler.fit_transform(bert_scores)
        zipf_scores_norm = zipf_scaler.fit_transform(zipf_scores)
        fasttext_scores_norm = fasttext_scaler.fit_transform(fasttext_scores)
        synonym_scores_norm = synonym_scaler.fit_transform(synonym_scores)

        for i in range(len(suggetions)):
            final_score = (bert_scores_norm[i] + zipf_scores_norm[i] + fasttext_scores_norm[i] + synonym_scores_norm[i]) / 4
            substitutions.append((suggetions[i]["token_str"], suggetions[i]["changed"], final_score))

        # sort suggestions by score and replace the word with the best suggestion if it has a higher Zipf score
        substitutions.sort(reverse=True, key=lambda x: x[2])
        #print(substitutions)

    return substitutions

In [241]:
def align_word_form(token, suggestion):
    #search if original word has inflections available for its' pos tag
    has_inflections = False
    inflection_ids = []

    #special case for participe verbs, take form after adjective
    if token.tag_[:3] == "Vmp":
        if token.lemma_ in word_inflections:
            if token.tag_ in spacy_tags:
                has_inflections = True
                inflection_ids = spacy_tags[token.tag_]["adj_id"]

        #take lemma as the one of the adjective
        suggestion_lemma = NLP(suggestion)[0].lemma_
        if suggestion_lemma in word_inflections:
            if "52" in word_inflections[suggestion_lemma]["inflections"]:
                suggestion_lemma = word_inflections[suggestion_lemma]["inflections"]["52"]
            
        if has_inflections and suggestion_lemma in word_inflections:
            for id in inflection_ids:
                str_id = str(id)
                if str_id in word_inflections[suggestion_lemma]["inflections"]:
                    if token.is_title:
                        return word_inflections[suggestion_lemma]["inflections"][str_id].title()
                    return word_inflections[suggestion_lemma]["inflections"][str_id]
        
        if token.is_title:
            return suggestion.title()
        return suggestion

    if token.lemma_ in word_inflections:
        if token.tag_ in spacy_tags:
            has_inflections = True
            inflection_ids = spacy_tags[token.tag_]["dex_online_id"]

    #replace the top replacement with the right form if found
    suggestion_lemma = NLP(suggestion)[0].lemma_

    if has_inflections and suggestion_lemma in word_inflections:
        for id in inflection_ids:
            str_id = str(id)
            if str_id in word_inflections[suggestion_lemma]["inflections"]:
                if token.is_title:
                    return word_inflections[suggestion_lemma]["inflections"][str_id].title()
                return word_inflections[suggestion_lemma]["inflections"][str_id]

    #if not replacement is found return original word
    if token.is_title:
        return suggestion.title()
    return suggestion

In [246]:
def sentence_simplification(sentence):
    token_replaced = True
    new_tokens = set([])

    #preprocessing
    sentence = preprocess_sentence(sentence)

    #sent_tokens = NLP(sentence)
    #for token in sent_tokens:
        #if token.ent_iob != 2:
            #print(token.text, token.ent_type_)
        #print(token.text, token.pos_, token.tag_)

    while token_replaced:
        token_replaced = False

        replacable_tokens = word_candidates(sentence, new_tokens)
        
        for token in replacable_tokens:
            if token_replaced:
                break

            token_text = token[0].text
            changed_sentence = sentence

            if token[0].pos_ == "NOUN":
                (suggestions, changed_sentence) = noun_substitution_generation(sentence, token[0])
            else:
                suggestions = substitution_generation(sentence, token[0])
            suggestions = substitution_ranking(suggestions, token[0])

            if len(suggestions) == 0:
                continue
            
            top_replacement = suggestions[0][0]

            if zipf_frequency(top_replacement, 'ro') >= zipf_frequency(token_text, 'ro'):
                top_replacement = align_word_form(token[0], top_replacement)
                if suggestions[0][1] == False:
                    sentence = sentence[:token[0].idx] + top_replacement + sentence[token[0].idx + len(token_text):]
                else:
                    sentence = changed_sentence.replace("[MASK]", top_replacement) 
                new_tokens.add(top_replacement)
                token_replaced = True

    return sentence

In [247]:
def text_simplification(text):
    doc = NLP(text)
    for sentence in doc.sents:
        new_sentence = sentence_simplification(sentence.text)
        text = text.replace(sentence.text, new_sentence)
    return text

In [248]:
#normal sentences
sentence_list = ["Priveliștea era spectaculoasă și se întindea până la orizont.", 
                 "Ei consumaseră prea mult alcool în acea seară.", 
                 "Mihai a mers la mall ca să își achiziționeze îmbrăcăminte.",
                 "Eu și gașca mea am mers în vacanță la litoral.",
                 "Am mâncat prea multă ciocolată și am făcut hiperglicemie.", 
                 "La facultate am studiat despre algoritmi și structuri de date.",
                 "Articolul publicat de mine a fost citit de toată lumea.",]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

Marea era spectaculoasă și se întindea până la orizont.
Ei avuseseră prea mult alcool în acea seară.
Mihai a mers la magazin ca să își cumpere haină.
Eu și familia mea am mers în vacanță la mare.
Am mâncat prea multă ciocolată și am făcut diabet.
La liceu am învățat despre structuri și structuri de date.
Articolul publicat de mine a fost citit de toată lumea.


In [249]:
#complex sentences
sentence_list = ["Procrastinarea constantă în fața responsabilităților academice poate duce la o exacerbare a anxietății și la compromiterea performanțelor intelectuale pe termen lung.",
                 "Implementarea unor strategii pedagogice inovatoare poate facilita internalizarea cunoștințelor teoretice într-un mod eficient și sustenabil.",
                 "Perturbarea echilibrului ecologic cauzată de defrișările necontrolate generează consecințe ireversibile asupra biodiversității și climatului global.",
                 "Diseminarea informațiilor eronate prin canale digitale poate conduce la dezinformare masivă și la erodarea încrederii publice în instituțiile fundamentale.",
                 "Complexitatea proceselor administrative influențează negativ eficiența deciziilor guvernamentale.",
                 "Polarizarea ideologică din societate poate compromite coeziunea comunitară și exacerba conflictele latente."]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

Creșterea constantă în fața sarcinilor științifice poate duce la o creștere a depresiei și la pierderea performanțelor intelectuale pe termen lung.
Dezvoltarea unor strategii didactice inovatoare poate facilita utilizarea cunoștințelor teoretice într-un mod eficient și durabil.
Pierderea echilibrului natural cauzată de copacii tăiați are consecințe grave asupra vieții și mediului global. 
Promovarea informațiilor eronate prin canale digitale poate conduce la manipulare masivă și la pierderea încrederii publice în instituțiile fundamentale.
Calitatea proceselor administrative influențează negativ eficiența deciziilor guvernamentale.
Conflictul politic din societate poate distruge unitatea națională și agrava conflictele existente. 


In [250]:
#special sentences
sentence_list = ["Au trecut peste 147 de ani de la înființarea Universității din București, iar eu am studiat acolo trei ani.", 
                 "Găsiți mai multe informații pe https://spacy.io/api/token.",]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

Au trecut peste 147 de ani de la înființarea Universității din București, iar eu am fost acolo trei ani.
Găsiți mai multe informații pe https://spacy.io/api/token.


In [251]:
text = "În contextul actual al dezvoltării societății contemporane, se observă o multitudine de problematici emergente care influențează semnificativ calitatea vieții cotidiene. Diversitatea factorilor determinanți contribuie la creșterea complexității soluțiilor propuse pentru ameliorarea acestor deficiențe. De exemplu, în domeniul educației, implementarea unor strategii pedagogice elaborate și utilizarea unor metodologii avansate au devenit priorități esențiale. Totuși, o parte considerabilă a populației rămâne marginalizată din cauza lipsei accesibilității resurselor educaționale. Pe de altă parte, în sectorul economic, dinamica pieței și globalizarea contribuie la apariția unor provocări substanțiale pentru întreprinderile mici și mijlocii. În acest sens, este imperativ să fie identificate modalități eficiente pentru stimularea sustenabilității acestor organizații. Un alt aspect relevant este cel legat de sănătate. Progresul tehnologic a facilitat crearea unor dispozitive medicale inovative, însă accesul la acestea este, din păcate, limitat pentru o parte semnificativă a populației."

print(text_simplification(text))

În contextul actual al dezvoltării societății moderne, se observă o multitudine de probleme majore care influențează semnificativ calitatea vieții cotidiene. Integrarea factorilor externi contribuie la creșterea calității soluțiilor propuse pentru rezolvarea acestor probleme. De exemplu, în domeniul educației, dezvoltarea unor metode didactice elaborate și utilizarea unor metode avansate au devenit obiective esențiale. Totuși, o parte mare a populației rămâne pierdută din cauza lipsei calității serviciilor sociale. Pe de altă parte, în sectorul economic, evoluția pieței și globalizarea contribuie la apariția unor probleme importante pentru întreprinderile mici și mici. În acest sens, este esențial să fie identificate soluții eficiente pentru creșterea eficienței acestor organizații. Un alt aspect important este cel legat de sănătate. Dezvoltarea economică a permis crearea unor dispozitive medicale noi, însă accesul la acestea este, din păcate, limitat pentru o parte semnificativă a pop

In [252]:
text = "România este un stat situat în sud-estul Europei Centrale, pe cursul inferior al Dunării, la nord de peninsula Balcanică și la țărmul nord-vestic al Mării Negre. " + "România este plasată geografic și în Europa de Est, Europa de Sud-Est respectiv parțial în Europa Centrală. Din punct de vedere geopolitic, România este un stat situat în Europa Centrală și de Est iar din punct de vedere cultural se încadrează parțial în conceptul de „Mitteleuropa” (i.e., Europa de mijloc sau Europa Centrală în limba germană) prin regiunile istorice Transilvania, Banat și Bucovina. Pe teritoriul ei este situată aproape toată suprafața Deltei Dunării și partea sudică și centrală a Munților Carpați. Se învecinează cu Bulgaria la sud, Serbia la sud-vest, Ungaria la nord-vest, Ucraina la nord și est și Republica Moldova la est, iar țărmul Mării Negre se găsește la sud-est." + "De-a lungul istoriei, diferite porțiuni ale teritoriului de astăzi al României au fost în componența sau sub administrația Daciei, grecilor pontici (e.g., Histria, Tomis și Callatis), Imperiului Persan, Imperiului Roman, goților, (mai precis vizigoților, i.e., Gutthiuda), gepizilor (i.e., Gepidia), hunilor, avarilor, cumanilor (i.e., Cumania), Imperiului Roman de Răsărit (sau Imperiul Bizantin), Țaratului Bulgar, Țării Românești, Republicii Genova (i.e., coloniile genoveze din România), Republicii Venețiene (i.e., anumite colonii comerciale precum San Giorgio/Giurgiu și Barilla/Brăila), Principatului Moldovei, Imperiului Otoman, Imperiului Rus, Imperiului Habsburgic, Imperiului Austriac și, respectiv, Austro-Ungariei. România a apărut ca stat, condus de Domnitorul Alexandru Ioan Cuza, în 1859, prin unirea dintre Moldova și Țara Românească, păstrând autonomia și statutul de stat tributar față de Imperiul Otoman, pe care-l aveau cele două principate. A fost recunoscută ca țară independentă 19 ani mai târziu. În 1918, în urma Primului Război Mondial, Transilvania, Banatul, Bucovina și Basarabia s-au unit cu Vechiul Regat formând România Mare, România dodoloață sau România interbelică, care a avut cea mai mare extindere teritorială din istoria țării (mai precis 295.641 km2). În timpul celui de-Al Doilea Război Mondial (în 1940), România Mare (i.e., Regatul României extins după 1918), sub presiunea Germaniei Naziste, a cedat teritorii Regatului Ungariei (mai precis nord-estul Transilvaniei), Regatului Bulgariei Bulgariei (Cadrilaterul sau Dobrogea de Sud) și Uniunii Sovietice (Basarabia, Herța și Bucovina de Nord). După abolirea dictaturii lui Antonescu la 23 august 1944, România s-a retras din alianța cu Puterile Axei, trecând de partea Puterilor Aliate (Regatul Unit, Statele Unite, Franța și Uniunea Sovietică). Prin Tratatul de pace de la Paris semnat la 10 februarie 1947, din teritoriile cedate ale fostei Românii Mari, a fost recuperată doar Transilvania de Nord. După înlăturarea regimului comunist instalat în România (1989) și după destrămarea Uniunii Sovietice (1991), statul a inițiat o serie de reforme economice și politice. După un deceniu de probleme economice, România a introdus noi reforme economice de ordin general (precum cota unică de impozitare, în 2005) și a aderat la alianța politico-militară NATO la 29 martie 2004 și la Uniunea Europeană la 1 ianuarie 2007."
print(text_simplification(text))


România este un stat situat în sud-estul Europei Centrale, pe cursul superior al Dunării, la nord de peninsula Balcanică și la malul nord-vestic al Mării Negre. România este plasată geografic și în Europa de Est, Europa de Sud-Est respectiv parțial în Europa Centrală. Din punct de vedere politic, România este un stat situat în Europa Centrală și de Est iar din punct de vedere cultural se încadrează parțial în conceptul de „Mitteleuropa” (i.e., Europa de mijloc sau Europa Centrală în limba germană) prin regiunile istorice Transilvania, Banat și Bucovina. Pe teritoriul ei este situată aproape toată suprafața Deltei Dunării și partea sudică și centrală a Munților Carpați. Se învecinează cu Bulgaria la sud, Serbia la sud-vest, Ungaria la nord-vest, Ucraina la nord și est și Republica Moldova la est, iar malul Mării Negre se găsește la sud-est.De-a lungul istoriei, diferite părți ale teritoriului de astăzi al României au fost în componența sau sub administrația Daciei, românilor romani (n.,

In [253]:
text = "În informatică, un procesor este un dispozitiv hardware al unui computer care pornind de la un set de instrucțiuni efectuează operațiuni pe o sursă externă de date. Termenul este frecvent utilizat pentru a face referire la unitatea centrală de procesare dintr-un sistem. Procesorul este elementul principal al unui sistem de calcul și încorporează funcțiile unității centrale de prelucrare a informației a unui calculator sau a unui sistem electronic structurat funcțional (care coordonează sistemul). De obicei, fizic procesorul se prezintă sub forma unui microprocesor, care este fabricat pe un singur cip de circuit integrat metal-oxid-semiconductor (MOS). Reprezintă forma structurală cea mai complexă pe care o pot avea circuitele integrate. Cipul semiconductor, care este plasat pe placa de bază, este foarte complex, putând ajunge să conțină milioane de microtranzistoare. El controlează activitățile întregului sistem în care este integrat și poate prelucra datele furnizate de utilizator. Procesorul asigură procesarea instrucțiunilor și datelor, atât din sistemul de operare al sistemului, cât și din aplicațiile utilizatorului, și anume le interpretează, prelucrează și controlează, execută sau supervizează transferurile de informații și controlează activitatea generală a celorlalte componente care alcătuiesc un sistem de calcul."
print(text_simplification(text))

În informatică, un procesor este un dispozitiv hardware al unui calculator care pornind de la un set de instrucțiuni efectuează operațiuni pe o sursă publică de date. Termenul este frecvent folosit pentru a face referire la unitatea centrală de producție dintr-un sistem. Sistemul este elementul principal al unui sistem de calcul și include funcțiile unității centrale de prelucrare a informației a unui calculator sau a unui sistem electronic structurat funcțional (care coordonează sistemul). De obicei, fizic procesorul se prezintă sub forma unui calculator, care este produs pe o singură placă de circuit integrată metal (MOS).  Reprezintă forma generală cea mai complexă pe care o pot avea sistemele integrate. Dispozitivul magnetic, care este plasat pe placa de bază, este foarte complex, putând ajunge să conțină milioane de componente. El controlează activitățile întregului sistem în care este integrat și poate utiliza datele furnizate de sistem. Sistemul asigură prelucrarea instrucțiunil